In [1]:
import pyvista
import torch
from diffdrr.drr import DRR
from diffdrr.visualization import drr_to_mesh, img_to_mesh

from diffpose.deepfluoro import DeepFluoroDataset
from diffpose.visualization import fiducials_to_mesh, lines_to_mesh

/home/ensai/anaconda3/envs/rxtool/lib/python3.11/site-packages/diffpose/calibration.py:23: UserWarning: As of jaxtyping version 0.2.24, jaxtyping now prefers the syntax
```
from jaxtyping import jaxtyped
# Use your favourite typechecker: usually one of the two lines below.
from typeguard import typechecked as typechecker
from beartype import beartype as typechecker

@jaxtyped(typechecker=typechecker)
def foo(...):
```
and the old double-decorator syntax
```
@jaxtyped
@typechecker
def foo(...):
```
should no longer be used. (It will continue to work as it did before, but the new approach will produce more readable error messages.)
In particular note that `typechecker` must be passed via keyword argument; the following is not valid:
```
@jaxtyped(typechecker)
def foo(...):
```

  @jaxtyped
/home/ensai/anaconda3/envs/rxtool/lib/python3.11/site-packages/diffpose/calibration.py:125: UserWarning: As of jaxtyping version 0.2.24, jaxtyping now prefers the syntax
```
from jaxtyping import jaxty

In [ ]:
# pyvista.start_xvfb()
# pyvista.global_theme.trame.server_proxy_enabled = True
# pyvista.global_theme.trame.server_proxy_prefix = "/proxy/"

In [2]:
pyvista.set_jupyter_backend("trame")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Initialize DiffDRR for Patient 1
SUBSAMPLE = 6.410714285714286
specimen = DeepFluoroDataset(id_number=1)
height = int((1536 - 100) / SUBSAMPLE)
delx = 0.194 * SUBSAMPLE

drr = DRR(
    specimen.volume,
    specimen.spacing,
    sdr=specimen.focal_len / 2,
    height=height,
    delx=delx,
    x0=specimen.x0,
    y0=specimen.y0,
    reverse_x_axis=True,
    bone_attenuation_multiplier=2.5,
).to(device)

In [4]:
_, pose = specimen[69]
rotations = pose.get_rotation().to(device)
translations = pose.get_translation().to(device)

In [6]:
# Extract a mesh from the CT
ct = drr_to_mesh(drr, method="marching_cubes", threshold=145, verbose=True)

# Make meshes for the camera and detector plane and
# convert the DRR into a texture for the detector plane
camera, detector, texture, principal_ray = img_to_mesh(
    drr, rotations, translations, "matrix"
)

# Compute the locations of 3D fiducials and projected 2D fiducials
fiducials_3d, fiducials_2d = fiducials_to_mesh(
    specimen,
    rotations,
    translations,
    detector=detector,
    parameterization="matrix",
)

# Draw lines from the camera to the 2D fiducials
lines = lines_to_mesh(camera, fiducials_2d)

Computing Contour: 100%|██████████[00:00<00:00]
Finding and Labeling Connected Regions.: 100%|██████████[00:00<00:00]
Smoothing Mesh using Taubin Smoothing: 100%|██████████[00:01<00:00]
Decimating Mesh: 100%|██████████[00:03<00:00]
Filling Holes: 100%|██████████[00:00<00:00]
Cleaning: 100%|██████████[00:00<00:00]


## Rendering a single X-ray in an interactive window

Exporting a notebook to HTML does not also automatically catch the PyVista plots, so the interactive plot has been manually saved to HTML and loaded.

In [7]:
plotter = pyvista.Plotter()
plotter.add_mesh(ct)
plotter.add_mesh(camera, show_edges=True)
plotter.add_mesh(detector, texture=texture)
plotter.add_mesh(principal_ray, color="red")
plotter.add_mesh(
    fiducials_3d,
    color="blueviolet",
    point_size=7.5,
    render_points_as_spheres=True,
)
plotter.add_mesh(
    fiducials_2d,
    color="lime",
    point_size=5,
    render_points_as_spheres=True,
)
for line in lines:
    plotter.add_mesh(line, color="lime")

plotter.add_axes()
plotter.add_bounding_box()

# plotter.show()  # If running Jupyter locally
# plotter.show(jupyter_backend="server")  # If running Jupyter remotely
plotter.export_html("render.html")

In [ ]:
from IPython.display import IFrame

IFrame("render.html", height=500, width=749)

## Rendering multiple X-rays in a static window

In [11]:
# Initialize the plot with the CT and 3D fiducials (shared across all plots)
plotter = pyvista.Plotter()
plotter.add_mesh(ct)
plotter.add_mesh(
    fiducials_3d,
    color="blueviolet",
    point_size=7.5,
    render_points_as_spheres=True,
)

# Render a subset of the X-rays
for idx, color in zip([0, 2, 69, 100], ["#1b9e77", "#d95f02", "#7570b3", "#e7298a"]):
    _, pose = specimen[idx]
    rotations = pose.get_rotation().to(device)
    translations = pose.get_translation().to(device)

    camera, detector, texture, _ = img_to_mesh(
        drr, rotations, translations, parameterization="matrix"
    )
    _, fiducials_2d = fiducials_to_mesh(
        specimen, rotations, translations, detector=detector, parameterization="matrix"
    )
    lines = lines_to_mesh(camera, fiducials_2d)

    plotter.add_mesh(camera, show_edges=True, line_width=3)
    plotter.add_mesh(detector, texture=texture)
    plotter.add_mesh(
        fiducials_2d,
        color=color,
        point_size=5,
        render_points_as_spheres=True,
    )
    for line in lines:
        plotter.add_mesh(line, color=color)

plotter.add_axes()
plotter.add_bounding_box()
plotter.show()

Widget(value='<iframe src="http://localhost:37239/index.html?ui=P_0x77471a524d90_2&reconnect=auto" class="pyvi…